In [1]:
from gensim.models import KeyedVectors 
import nltk
import string
import spacy
import os
from operator import itemgetter
import networkx as nx
import xml.etree.ElementTree as ET
nlp = spacy.load('en_core_web_sm')

In [2]:
wv = KeyedVectors.load_word2vec_format('C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\Libretas\\asoc.emb',binary=False)
print('Modelo cargado...')
total = wv['insect'] + wv['honey']
print(wv.most_similar(total,topn=20))
wv = KeyedVectors.load_word2vec_format('C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\Libretas\\freq.emb',binary=False)
print('Modelo cargado...')
print(wv.most_similar(total,topn=20))


Modelo cargado...
[('honey', 0.7876840829849243), ('insect', 0.753071129322052), ('insects', 0.617685854434967), ('buzzing', 0.5793455839157104), ('bumble-bee', 0.5287588238716125), ('arthropod', 0.5233635902404785), ('dragonfly', 0.5055277943611145), ('tsetse', 0.49677738547325134), ('bees', 0.4962603449821472), ('creepy_crawlies', 0.4921882152557373), ('bee', 0.49186939001083374), ('antennae', 0.48447319865226746), ('molten', 0.47563406825065613), ('hive', 0.4716135859489441), ('crawlies', 0.45693933963775635), ('mosquito', 0.45368626713752747), ('crawly', 0.4534505605697632), ('caterpillar', 0.45209571719169617), ('ger', 0.45130759477615356), ('bug', 0.44963157176971436)]
Modelo cargado...
[('bromide', 0.3149372935295105), ('char', 0.3028585612773895), ('injures', 0.29937535524368286), ('stabs', 0.29265496134757996), ('murders', 0.29152122139930725), ('cuckold', 0.2840166687965393), ('sigmund', 0.2812584936618805), ('cupid', 0.27586665749549866), ('paler', 0.27491000294685364), ('si

In [12]:
wv = KeyedVectors.load_word2vec_format('C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\Libretas\\asocLem.emb',binary=False)
print('Modelo cargado...')
total = wv['insect'] + wv['honey']
print(wv.most_similar(total,topn=20))

Modelo cargado...
[('honey', 0.8095800876617432), ('insect', 0.7649351358413696), ('buzzing', 0.5684378147125244), ('mosquitoes', 0.5374653935432434), ('hive', 0.5151685476303101), ('antennae', 0.5132050514221191), ('creepy_crawlies', 0.5021637082099915), ('bees', 0.48864299058914185), ('beetles', 0.48434269428253174), ('arthropod', 0.48424509167671204), ('dragonfly', 0.47871631383895874), ('tsetse', 0.46332478523254395), ('insects', 0.46269744634628296), ('bee', 0.4597940146923065), ('swarms', 0.45840346813201904), ('feelers', 0.45523738861083984), ('mosquito', 0.45478135347366333), ('ger', 0.4463757574558258), ('crawlies', 0.4450555145740509), ('crawly', 0.44468024373054504)]


In [3]:
def conceptos(lista,subconjunto):
    datos = []
    if len(lista) <= 100:
        tope = len(lista)
    else:
        tope = 100
    for x in range(0,tope):
        if str(lista[x][0]) not in subconjunto and float(lista[x][1])>0:
            datos.append(str(lista[x][0]))
    return datos

In [4]:
def precision(k,arreglo,concepto):
    tope = k
    encontrado = 0
    if k>len(arreglo):
        tope = len(arreglo)
    for i in range(0,tope):
        if arreglo[i] == concepto.strip():
            encontrado = 1
    return encontrado

In [5]:
def limpia_lematiza(cadena):
    limpiado = ''
    palabras_funcionales = nltk.corpus.stopwords.words('english')
    for c in string.punctuation:
        cadena = cadena.replace(c,"")
    cadena = cadena.strip()
    for palabra in cadena.split(" "):
        if palabra not in palabras_funcionales:
            doc = nlp(palabra)
            if len(doc)>0:
                limpiado = limpiado + doc[0].lemma_ + " "
    return limpiado

In [6]:
def diccionario_vectorial(wv,definicion):
    texto = limpia_lematiza(definicion)
    texto = texto.lower()
    tokens = texto.split(" ")
    subconjunto_lemas = []
    suma = 0
    for palabra in tokens:
        if palabra in wv and palabra != '':
            subconjunto_lemas.append(palabra)
    if len(subconjunto_lemas) > 0:
        for palabra in subconjunto_lemas:
            suma += wv[palabra]
        resultados = wv.most_similar(suma,topn=20)
        return conceptos(resultados,subconjunto_lemas)
    else:
        return []

In [21]:
def rendimiento_mdbl_vectorial():
    wv = KeyedVectors.load_word2vec_format('C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\Libretas\\freq.emb',binary=False)
    print('Modelo cargado...') 
    p_1 = p_3 = p_5 = p_10 = 0
    total = 0    
    directorio = "C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\concepts\\"
    for archivo in os.listdir(directorio):
        datos = open(directorio+archivo,encoding="utf8")
        lineas = datos.readlines()
        concepto = archivo[0:archivo.find('.txt')]
        for linea in lineas:
            total += 1
            candidatos = diccionario_vectorial(wv,linea)
            p_1 += precision(1,candidatos,concepto)
            p_3 += precision(3,candidatos,concepto)
            p_5 += precision(5,candidatos,concepto)
            p_10 += precision(10,candidatos,concepto)
    print("Precisión")
    print("p@1:",p_1/total,"p@3",p_3/total,"p@5",p_5/total,"p@10",p_10/total)
    return

## Modelo basado en asociación sin lematizar

In [18]:
rendimiento_mdbl_vectorial()

Modelo cargado...
Precisión
p@1: 0.22857142857142856 p@3 0.37142857142857144 p@5 0.4142857142857143 p@10 0.4714285714285714


## Modelo basado en asociación lematizado

In [20]:
rendimiento_mdbl_vectorial()

Modelo cargado...
Precisión
p@1: 0.18571428571428572 p@3 0.2857142857142857 p@5 0.32857142857142857 p@10 0.4


## Modelo basado en frecuencia sin lematizar

In [22]:
rendimiento_mdbl_vectorial()

Modelo cargado...
Precisión
p@1: 0.04285714285714286 p@3 0.14285714285714285 p@5 0.22857142857142856 p@10 0.24285714285714285


In [20]:
def lee_xml_eat():
    lista_grafos = []
    G_freq = nx.Graph()
    G_asoc = nx.Graph()
    tree = ET.parse("eat-stimulus-response.xml") 
    root = tree.getroot()
    i = 0
    for child in root:
        estimulo = child.get('word')
        total = child.get('all')
        estimulo = estimulo.lower()
        estimulo = estimulo.replace(" ","_")
        i+=1
        #print(i)
        for response in child.iter('response'):
            frecuencia = response.get('n')
            asociacion = response.get('r')
            respuesta = response.get('word')
            respuesta = respuesta.lower()
            respuesta = respuesta.replace(' ','_')
            G_freq.add_edge(estimulo,respuesta,weight=float(1/int(frecuencia)))
            G_asoc.add_edge(estimulo,respuesta,weight=float(1-float(asociacion)))
    lista_grafos.append(G_freq)
    lista_grafos.append(G_asoc)
    print("Grafos creados. Nodos:",len(nx.nodes(G_freq)))
    return lista_grafos

In [22]:
def conceptos_nuevo_orden(lista,subconjunto,grafo):
    datos = []
    if len(lista) <= 100:
        tope = len(lista)
    else:
        tope = 100
    print(lista)
    nueva_lista = [(elemento[0],elemento[1]*grafo.degree(elemento[0])) for elemento in lista]
    lista = sorted(nueva_lista,key=itemgetter(1),reverse=True)
    print(lista)
    for x in range(0,tope):
        if str(lista[x][0]) not in subconjunto and float(lista[x][1])>0:
            datos.append(str(lista[x][0]))
    return datos

In [11]:
def diccionario_hibrido(wv,grafo,definicion):
    texto = limpia_lematiza(definicion)
    texto = texto.lower()
    tokens = texto.split(" ")
    subconjunto_lemas = []
    suma = 0
    for palabra in tokens:
        if palabra in wv and palabra != '':
            subconjunto_lemas.append(palabra)
    if len(subconjunto_lemas) > 0:
        for palabra in subconjunto_lemas:
            suma += wv[palabra]
        resultados = wv.most_similar(suma,topn=20)
        return conceptos_nuevo_orden(resultados,subconjunto_lemas,grafo)
    else:
        return []

In [25]:
def rendimiento_mdbl_hibrido():
    wv = KeyedVectors.load_word2vec_format('C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\Libretas\\freq.emb',binary=False)
    grafos = lee_xml_eat()
    grafo_asoc = grafos[1]
    print('Modelo cargado...') 
    p_1 = p_3 = p_5 = p_10 = 0
    total = 0    
    directorio = "C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\concepts\\"
    for archivo in os.listdir(directorio):
        datos = open(directorio+archivo,encoding="utf8")
        lineas = datos.readlines()
        concepto = archivo[0:archivo.find('.txt')]
        for linea in lineas:
            total += 1
            candidatos = diccionario_hibrido(wv,grafo_asoc,linea)
            p_1 += precision(1,candidatos,concepto)
            p_3 += precision(3,candidatos,concepto)
            p_5 += precision(5,candidatos,concepto)
            p_10 += precision(10,candidatos,concepto)
    print("Precisión")
    print("p@1:",p_1/total,"p@3",p_3/total,"p@5",p_5/total,"p@10",p_10/total)
    return

## Modelo basado en embeddings de asociación & grados del nodo

In [24]:
rendimiento_mdbl_hibrido()

Grafos creados. Nodos: 23218
Modelo cargado...
[('sit', 0.5581364035606384), ('wood', 0.5560597777366638), ('park', 0.5273075699806213), ('street', 0.4967517852783203), ('you', 0.47538328170776367), ('make', 0.46424397826194763), ('stile', 0.4095579981803894), ('footstool', 0.40243837237358093), ('grained', 0.40133237838745117), ('nearside', 0.3957040309906006), ('environs', 0.3937976062297821), ('stroll', 0.3921579420566559), ('sat', 0.3916930556297302), ('of_heaven', 0.3889417052268982), ('leafy', 0.38871562480926514), ('thirty-nine', 0.38665202260017395), ('seat', 0.3861575126647949), ('easier', 0.3725590109825134), ('furniture', 0.37232863903045654), ('lane', 0.37223562598228455)]
[('wood', 191.28456354141235), ('you', 168.7610650062561), ('street', 81.46729278564453), ('make', 72.88630458712578), ('park', 63.27690839767456), ('sit', 48.55786710977554), ('seat', 44.79427146911621), ('lane', 30.151085704565048), ('furniture', 26.063004732131958), ('footstool', 22.938987225294113), (

[('use', 0.6414355039596558), ('cleanly', 0.5261948704719543), ('it', 0.5138356685638428), ('utility', 0.5129730701446533), ('something', 0.46514928340911865), ('lieu', 0.46357110142707825), ('spade', 0.4558461308479309), ('utilize', 0.451450914144516), ('tu_brute', 0.44368666410446167), ('also', 0.4430164694786072), ('as_what', 0.44219037890434265), ('in_place_of', 0.4400624930858612), ("what's", 0.43928515911102295), ('a', 0.43903830647468567), ('than', 0.4350108802318573), ('thing', 0.4341796040534973), ('like', 0.4331626296043396), ('make', 0.4227466583251953), ('as_can_be_expected', 0.422607958316803), ('contain', 0.42219626903533936)]
[('it', 172.13494896888733), ('like', 84.89987540245056), ('thing', 83.36248397827148), ('use', 73.76508295536041), ('make', 66.37122535705566), ('something', 59.07395899295807), ('a', 38.19633266329765), ('than', 37.41093569993973), ('spade', 32.820921421051025), ('also', 25.25193876028061), ('contain', 24.487383604049683), ("what's", 10.5428438186

[('something', 0.6287457346916199), ('use', 0.578394889831543), ('carry', 0.5678086280822754), ('container', 0.5418005585670471), ('a', 0.5246909260749817), ('sort', 0.5187416076660156), ('contain', 0.5050835609436035), ('utility', 0.468282014131546), ('freight_liner', 0.45896923542022705), ('in_place_of', 0.4571971297264099), ('any', 0.4517707824707031), ('cleanly', 0.440712034702301), ('other', 0.4255426526069641), ('in_kind', 0.4233361482620239), ('lieu', 0.4223489463329315), ('camion', 0.40444064140319824), ('can', 0.40252456068992615), ('it', 0.40192344784736633), ('freight', 0.4002682864665985), ('brought', 0.40008068084716797)]
[('it', 134.64435502886772), ('something', 79.85070830583572), ('use', 66.51541233062744), ('can', 52.3281928896904), ('a', 45.64811056852341), ('carry', 44.28907299041748), ('container', 37.9260390996933), ('other', 31.490156292915344), ('sort', 30.087013244628906), ('any', 29.816871643066406), ('contain', 29.294846534729004), ('brought', 23.604760169982

[('severe', 0.5130683779716492), ('lot', 0.5051829218864441), ('storm', 0.5001223683357239), ('wind', 0.47672274708747864), ('strong', 0.459291011095047), ('involve', 0.4501555562019348), ('kind', 0.4383927881717682), ('fierce', 0.4337940514087677), ('gusts', 0.42921897768974304), ('caning', 0.4193316102027893), ('typhoon', 0.40954917669296265), ('a', 0.4065737724304199), ('storms', 0.4049862027168274), ('vindictive', 0.39613285660743713), ('hurricanes', 0.3960396349430084), ('vane', 0.39598342776298523), ('mikado', 0.38972389698028564), ('aggressive', 0.3894130289554596), ('gentle', 0.3873478174209595), ('calm', 0.3848750591278076)]
[('wind', 106.78589534759521), ('strong', 87.26529210805893), ('kind', 69.26606053113937), ('calm', 48.49425745010376), ('storm', 48.01174736022949), ('lot', 46.9820117354393), ('gentle', 35.63599920272827), ('a', 35.37191820144653), ('severe', 29.757965922355652), ('fierce', 29.497995495796204), ('aggressive', 28.42715111374855), ('storms', 20.24931013584

[('citrus', 0.698920726776123), ('bitter', 0.6859340071678162), ('lemon', 0.6407070159912109), ('bitters', 0.620715320110321), ('fruit', 0.6111521124839783), ('lemons', 0.6019413471221924), ('sour', 0.5944165587425232), ('squeezer', 0.591514527797699), ('grapefruit', 0.5866882801055908), ('apricot', 0.5761152505874634), ('angostura', 0.5742243528366089), ('juice', 0.5678008794784546), ('melon', 0.5667001008987427), ('and_lemons', 0.5598247647285461), ('taste', 0.5546409487724304), ('tang', 0.5511659383773804), ('light_ale', 0.5374120473861694), ('tasteful', 0.5370460748672485), ('slice', 0.5344845056533813), ('ascorbic_acid', 0.5297343730926514)]
[('fruit', 122.84157460927963), ('taste', 68.77547764778137), ('bitter', 63.10592865943909), ('lemon', 50.615854263305664), ('juice', 47.12747299671173), ('sour', 40.42032599449158), ('tang', 28.66062879562378), ('melon', 27.76830494403839), ('grapefruit', 23.467531204223633), ('bitters', 21.725036203861237), ('slice', 20.31041121482849), ('ap

[('citrus', 0.8384882211685181), ('fruit', 0.7051693201065063), ('yellow', 0.696841835975647), ('lemons', 0.6346805095672607), ('lemon', 0.6289379596710205), ('squeezer', 0.6230605244636536), ('grapefruit', 0.5810697674751282), ('oranges', 0.5775240659713745), ('juice', 0.5645571947097778), ('and_lemons', 0.5563814640045166), ('juices', 0.5522041916847229), ('bitters', 0.531811535358429), ('sour', 0.5287215709686279), ('fructose', 0.5219218134880066), ('fruits', 0.5188122987747192), ('jaffa', 0.5166479349136353), ('melon', 0.5158984661102295), ('orange', 0.5131091475486755), ('apricot', 0.5121190547943115), ('peaches', 0.5089411735534668)]
[('yellow', 154.69888758659363), ('fruit', 141.73903334140778), ('orange', 89.79410082101822), ('lemon', 49.68609881401062), ('juice', 46.85824716091156), ('sour', 35.9530668258667), ('oranges', 33.49639582633972), ('melon', 25.279024839401245), ('grapefruit', 23.242790699005127), ('juices', 20.431555092334747), ('lemons', 19.040415287017822), ('bitt

[('rodent', 0.6087719798088074), ('mammal', 0.6082011461257935), ('mole', 0.4946672320365906), ('reptile', 0.47764161229133606), ('grey', 0.470765084028244), ('countryside', 0.44425836205482483), ('alligator', 0.44412142038345337), ('town', 0.4418680667877197), ('shrew', 0.4380044937133789), ('small', 0.4376644790172577), ('catcher', 0.4375848174095154), ('guinea-pig', 0.43536925315856934), ('molehill', 0.42964720726013184), ('animal', 0.42014428973197937), ('tsetse', 0.41958504915237427), ('otter', 0.4149874746799469), ('elizabeth_taylor', 0.4063287079334259), ('creature', 0.40567171573638916), ('guinea_pig', 0.4032784104347229), ('furry', 0.3990163505077362)]
[('small', 121.23306068778038), ('animal', 119.74112257361412), ('town', 88.37361335754395), ('grey', 86.6207754611969), ('countryside', 46.20286965370178), ('creature', 32.048065543174744), ('furry', 29.52720993757248), ('mole', 28.690699458122253), ('otter', 20.749373733997345), ('shrew', 16.6441707611084), ('reptile', 15.7621

In [26]:
rendimiento_mdbl_hibrido()

Grafos creados. Nodos: 23218
Modelo cargado...
[('sit', 0.5479627847671509), ('park', 0.5323521494865417), ('wood', 0.5157992243766785), ('make', 0.4757736921310425), ('a_time', 0.47299349308013916), ('street', 0.4635830521583557), ('you', 0.4559793174266815), ('nisi', 0.4491417407989502), ('overruled', 0.41643086075782776), ('sat', 0.4149370789527893), ('maugham', 0.4109590947628021), ('disarrange', 0.404710590839386), ('thant', 0.4019733667373657), ('come_near', 0.39973095059394836), ('minster', 0.39597368240356445), ('crany', 0.3898804187774658), ('mar', 0.38497194647789), ('proximity', 0.3840715289115906), ('pointers', 0.38269639015197754), ('thrice', 0.37955713272094727)]
[('wood', 177.4349331855774), ('you', 161.87265768647194), ('street', 76.02762055397034), ('make', 74.69646966457367), ('park', 63.88225793838501), ('sit', 47.67276227474213), ('sat', 16.182546079158783), ('mar', 0.76994389295578), ('proximity', 0.7681430578231812), ('a_time', 0.47299349308013916), ('nisi', 0.449

[('use', 0.6030156016349792), ('lieu', 0.5154532194137573), ('utility', 0.4860312044620514), ('cleanly', 0.480381041765213), ('shalt', 0.4633297622203827), ('or_other', 0.4623925983905792), ('whosoever', 0.4598318636417389), ('it', 0.4545031487941742), ('commission', 0.4517979621887207), ('back_at_the_ranch', 0.4506484270095825), ('trebles', 0.4473295211791992), ('wherever', 0.44563934206962585), ('alkalis', 0.4446316361427307), ('widens', 0.44451600313186646), ('cetera', 0.44192829728126526), ('urged', 0.4410412013530731), ('sand', 0.4405677616596222), ('thing', 0.4403958320617676), ('thant', 0.44020378589630127), ('one_another', 0.440153032541275)]
[('it', 152.25855484604836), ('thing', 84.55599975585938), ('sand', 70.93140962719917), ('use', 69.34679418802261), ('utility', 0.9720624089241028), ('cleanly', 0.960762083530426), ('commission', 0.9035959243774414), ('lieu', 0.5154532194137573), ('shalt', 0.4633297622203827), ('or_other', 0.4623925983905792), ('whosoever', 0.4598318636417

[('and_tear', 0.6108392477035522), ('wear', 0.5752294063568115), ('shirt', 0.5518906712532043), ('repaired', 0.5281002521514893), ('trouser', 0.518011748790741), ('denim', 0.5171852111816406), ('ablative', 0.49194133281707764), ('dress', 0.478760689496994), ('bentos', 0.4741246998310089), ('hacking', 0.4740600883960724), ('unseemly', 0.47332820296287537), ('jumper', 0.46929416060447693), ('mutated', 0.4654621481895447), ('pullovers', 0.4622567892074585), ('garment', 0.4613029360771179), ('styled', 0.4596976637840271), ('wherever', 0.45937275886535645), ('whosoever', 0.45568734407424927), ('unwrap', 0.4553913176059723), ('sewn', 0.450693279504776)]
[('dress', 110.11495858430862), ('shirt', 72.29767793416977), ('wear', 41.99174666404724), ('jumper', 26.749767154455185), ('garment', 24.44905561208725), ('trouser', 19.166434705257416), ('sewn', 0.901386559009552), ('and_tear', 0.6108392477035522), ('repaired', 0.5281002521514893), ('denim', 0.5171852111816406), ('ablative', 0.4919413328170

[('first', 0.6038835644721985), ('name', 0.5870749354362488), ('viii', 0.5341181755065918), ('liver_salts', 0.47141674160957336), ('whatsit', 0.44972801208496094), ('wherever', 0.4485371708869934), ('ablative', 0.4464173913002014), ('no_see', 0.4463394582271576), ('thant', 0.44426262378692627), ('hur', 0.4410249590873718), ('trillion', 0.440966933965683), ('thrice', 0.4360446631908417), ('de_milo', 0.4356192350387573), ('other', 0.4347781836986542), ('lief', 0.4329901337623596), ('one', 0.4326276183128357), ('luther_king', 0.43198543787002563), ('back_at_the_ranch', 0.4264256954193115), ('kith', 0.4244955778121948), ('peters', 0.42435693740844727)]
[('name', 99.21566408872604), ('one', 94.31282079219818), ('first', 77.2970962524414), ('other', 32.17358559370041), ('viii', 0.5341181755065918), ('liver_salts', 0.47141674160957336), ('whatsit', 0.44972801208496094), ('wherever', 0.4485371708869934), ('ablative', 0.4464173913002014), ('no_see', 0.4463394582271576), ('thant', 0.444262623786

[('citrus', 0.7741349935531616), ('yellow', 0.6324287056922913), ('fruit', 0.5827134251594543), ('lemon', 0.5543794631958008), ('and_lemons', 0.54449862241745), ('grapefruit', 0.5432440042495728), ('oranges', 0.5395663380622864), ('angostura', 0.5380306839942932), ('squeezer', 0.5376542210578918), ('a', 0.529052734375), ('cyprus', 0.4992769658565521), ('lemons', 0.49396976828575134), ('alkalis', 0.49387332797050476), ('bitters', 0.4895574748516083), ('juices', 0.4822927415370941), ('jaffa', 0.4817512333393097), ('juice', 0.4767191708087921), ('milds', 0.46500396728515625), ('syrup', 0.46305394172668457), ('fructose', 0.4595952033996582)]
[('yellow', 140.39917266368866), ('fruit', 117.12539845705032), ('a', 46.027587890625), ('lemon', 43.79597759246826), ('juice', 39.567691177129745), ('oranges', 31.29484760761261), ('grapefruit', 21.72976016998291), ('syrup', 21.30048131942749), ('juices', 17.844831436872482), ('bitters', 17.13451161980629), ('lemons', 14.81909304857254), ('citrus', 13

[('tail', 0.6285114884376526), ('bushy', 0.6059712171554565), ('animal', 0.6020923256874084), ('tree', 0.5160315632820129), ('guinea-pig', 0.5018100142478943), ('nuts', 0.48223623633384705), ('squirrel', 0.45697200298309326), ('two_cities', 0.44623497128486633), ('fox', 0.44554847478866577), ('primates', 0.44459158182144165), ('live', 0.44385984539985657), ('preying', 0.44073250889778137), ('baskerville', 0.43795299530029297), ('and_bolts', 0.4329722225666046), ('kernel', 0.42593440413475037), ('d.h.lawrence', 0.42524293065071106), ('stalks', 0.4223543405532837), ('verdant', 0.42187634110450745), ('alligator', 0.42044928669929504), ('seeks', 0.41801518201828003)]
[('tree', 178.03088933229446), ('animal', 171.5963128209114), ('nuts', 70.40649050474167), ('tail', 65.36519479751587), ('live', 61.252658665180206), ('fox', 41.88155663013458), ('stalks', 23.651843070983887), ('squirrel', 20.106768131256104), ('bushy', 1.8179136514663696), ('kernel', 1.277803212404251), ('two_cities', 0.89246

[('h2o', 0.7291957139968872), ('liquid', 0.697989284992218), ('clear', 0.6520907282829285), ('h2so4', 0.6172860264778137), ('rapier', 0.6046242117881775), ('m.1.', 0.488167941570282), ('hcl', 0.4712717831134796), ('runny', 0.46912461519241333), ('alkalis', 0.46673882007598877), ('whisky', 0.4569964110851288), ('water_down', 0.45543745160102844), ('lava', 0.4492661952972412), ('nutrition', 0.4463790953159332), ('35', 0.44420960545539856), ('alcohol', 0.44119006395339966), ('chemical', 0.4397971034049988), ('viable', 0.43422016501426697), ('chemistry', 0.433973103761673), ('thirst', 0.43194177746772766), ('opaque', 0.42914557456970215)]
[('clear', 94.55315560102463), ('chemistry', 67.26583108305931), ('liquid', 67.00697135925293), ('whisky', 51.183598041534424), ('chemical', 42.22052192687988), ('alcohol', 37.50115543603897), ('thirst', 22.892914205789566), ('opaque', 18.882405281066895), ('h2o', 16.771501421928406), ('m.1.', 3.417175590991974), ('35', 2.221048027276993), ('hcl', 1.88508

In [9]:
def rendimiento_mdbl_vectorial():
    wv = KeyedVectors.load_word2vec_format('C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\Libretas\\dem_cbow.emb',binary=False)
    print('Modelo cargado...') 
    p_1 = p_3 = p_5 = p_10 = 0
    total = 0    
    directorio = "C:\\Users\\jorge.reyes\\Dropbox\\FMAT\\Agosto - Diciembre 2021\\PLN\\concepts\\"
    for archivo in os.listdir(directorio):
        datos = open(directorio+archivo,encoding="utf8")
        lineas = datos.readlines()
        concepto = archivo[0:archivo.find('.txt')]
        for linea in lineas:
            total += 1
            candidatos = diccionario_vectorial(wv,linea)
            p_1 += precision(1,candidatos,concepto)
            p_3 += precision(3,candidatos,concepto)
            p_5 += precision(5,candidatos,concepto)
            p_10 += precision(10,candidatos,concepto)
    print("Precisión")
    print("p@1:",p_1/total,"p@3",p_3/total,"p@5",p_5/total,"p@10",p_10/total)
    return

In [10]:
rendimiento_mdbl_vectorial()

Modelo cargado...
Precisión
p@1: 0.0 p@3 0.0 p@5 0.0 p@10 0.0
